In [5]:
from bs4 import BeautifulSoup
import pandas as pd

In [6]:
train = pd.read_csv("labeledTrainData.tsv",sep="\t")

In [8]:
test = pd.read_csv("testData.tsv",sep="\t")

In [12]:
unlabeled_train = pd.read_csv( "unlabeledTrainData.tsv", header=0, 
 delimiter="\t", quoting=3 )


In [16]:
import re
import nltk.data

In [3]:
from nltk.corpus import stopwordswords

In [41]:
def review_to_wordlist( review, remove_stopwords=False ):
    # Function to convert a document to a sequence of words,
    # optionally removing stop words.  Returns a list of words.
    #
    # 1. Remove HTML
    review_text = BeautifulSoup(review).get_text()
    #  
    # 2. Remove non-letters
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    #
    # 3. Convert words to lower case and split them
    words = review_text.lower().split()
    #
    # 4. Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    #
    # 5. Return a list of words
    return(words)
    
    
    
    

In [17]:
#Load the punkt tokenizer
tokenizer = nltk.data.load("tokenizers/punkt/english.pickle")

In [18]:
def review_to_sentences(review, tokenizer, remove_stopwords=False):
    # use the NLTK tokenizer to spilt the paragraph into sentences
    raw_sentences = tokenizer.tokenize(review.strip())
    
    sentences = []
    for raw_sentence in raw_sentences:
        if len(raw_sentence) > 0:
            sentences.append(review_to_wordlist(raw_sentence, remove_stopwords))
            
    return sentences
    

In [33]:
train["review"][1]

'\\The Classic War of the Worlds\\" by Timothy Hines is a very entertaining film that obviously goes to great effort and lengths to faithfully recreate H. G. Wells\' classic book. Mr. Hines succeeds in doing so. I, and those who watched his film with me, appreciated the fact that it was not the standard, predictable Hollywood fare that comes out every year, e.g. the Spielberg version with Tom Cruise that had only the slightest resemblance to the book. Obviously, everyone looks for different things in a movie. Those who envision themselves as amateur \\"critics\\" look only to criticize everything they can. Others rate a movie on more important bases,like being entertained, which is why most people never agree with the \\"critics\\". We enjoyed the effort Mr. Hines put into being faithful to H.G. Wells\' classic novel, and we found it to be very entertaining. This made it easy to overlook what the \\"critics\\" perceive to be its shortcomings."'

In [19]:
sentences = []

In [22]:
for review in train["review"]:
    sentences+=review_to_sentences(review, tokenizer)

C:\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))
C:\Anaconda3\lib\site-packages\bs4\__init__.py:219: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
C:\Anaconda3\lib\site-packages\bs4\__init__.py:219: UserWarning: "b'...'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup

In [42]:
for review in unlabeled_train["review"]:
    sentences += review_to_sentences(review, tokenizer)

C:\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))
C:\Anaconda3\lib\site-packages\bs4\__init__.py:219: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
C:\Anaconda3\lib\site-packages\bs4\__init__.py:282: UserWarning: "http://www.archive.org/details/LovefromaStranger"" looks like a URL. Beautiful Soup is not an HTTP client. You should pro

In [45]:
len(sentences)

795872

In [47]:
# Import the built-in logging module and configure it so that Word2Vec 
# creates nice output messages
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

# Set values for various parameters
num_features = 300    # Word vector dimensionality                      
min_word_count = 40   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

# Initialize and train the model (this will take some time)
from gensim.models import word2vec
print ("Training model...")
model = word2vec.Word2Vec(sentences, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)

# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)

# It can be helpful to create a meaningful model name and 
# save the model for later use. You can load it later using Word2Vec.load()
model_name = "300features_40minwords_10context"
model.save(model_name)

C:\Anaconda3\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
2018-03-02 20:47:17,900 : INFO : 'pattern' package not found; tag filters are not available for English
2018-03-02 20:47:17,920 : INFO : collecting all words and their counts
2018-03-02 20:47:17,921 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-03-02 20:47:18,060 : INFO : PROGRESS: at sentence #10000, processed 225664 words, keeping 17775 word types


Training model...


2018-03-02 20:47:18,185 : INFO : PROGRESS: at sentence #20000, processed 451582 words, keeping 24944 word types
2018-03-02 20:47:18,298 : INFO : PROGRESS: at sentence #30000, processed 670632 words, keeping 30023 word types
2018-03-02 20:47:18,434 : INFO : PROGRESS: at sentence #40000, processed 896478 words, keeping 34329 word types
2018-03-02 20:47:18,616 : INFO : PROGRESS: at sentence #50000, processed 1115469 words, keeping 37741 word types
2018-03-02 20:47:18,739 : INFO : PROGRESS: at sentence #60000, processed 1336692 words, keeping 40702 word types
2018-03-02 20:47:18,872 : INFO : PROGRESS: at sentence #70000, processed 1559365 words, keeping 43300 word types
2018-03-02 20:47:18,964 : INFO : PROGRESS: at sentence #80000, processed 1778623 words, keeping 45699 word types
2018-03-02 20:47:19,044 : INFO : PROGRESS: at sentence #90000, processed 2002603 words, keeping 48113 word types
2018-03-02 20:47:19,114 : INFO : PROGRESS: at sentence #100000, processed 2224101 words, keeping 50

2018-03-02 20:47:25,021 : INFO : PROGRESS: at sentence #740000, processed 16546073 words, keeping 119639 word types
2018-03-02 20:47:25,105 : INFO : PROGRESS: at sentence #750000, processed 16763375 words, keeping 120274 word types
2018-03-02 20:47:25,181 : INFO : PROGRESS: at sentence #760000, processed 16982901 words, keeping 120901 word types
2018-03-02 20:47:25,260 : INFO : PROGRESS: at sentence #770000, processed 17209327 words, keeping 121680 word types
2018-03-02 20:47:25,337 : INFO : PROGRESS: at sentence #780000, processed 17440408 words, keeping 122381 word types
2018-03-02 20:47:25,419 : INFO : PROGRESS: at sentence #790000, processed 17666965 words, keeping 123048 word types
2018-03-02 20:47:25,470 : INFO : collected 123504 word types from a corpus of 17798082 raw words and 795872 sentences
2018-03-02 20:47:25,471 : INFO : Loading a fresh vocabulary
2018-03-02 20:47:25,589 : INFO : min_count=40 retains 16490 unique words (13% of original 123504, drops 107014)
2018-03-02 20:

2018-03-02 20:48:31,970 : INFO : PROGRESS: at 47.29% examples, 457778 words/s, in_qsize 8, out_qsize 0
2018-03-02 20:48:32,980 : INFO : PROGRESS: at 48.12% examples, 458896 words/s, in_qsize 7, out_qsize 0
2018-03-02 20:48:33,989 : INFO : PROGRESS: at 48.86% examples, 459031 words/s, in_qsize 6, out_qsize 1
2018-03-02 20:48:34,995 : INFO : PROGRESS: at 49.62% examples, 459378 words/s, in_qsize 7, out_qsize 0
2018-03-02 20:48:36,003 : INFO : PROGRESS: at 50.32% examples, 459208 words/s, in_qsize 8, out_qsize 0
2018-03-02 20:48:37,033 : INFO : PROGRESS: at 51.10% examples, 459593 words/s, in_qsize 8, out_qsize 0
2018-03-02 20:48:38,040 : INFO : PROGRESS: at 51.85% examples, 459905 words/s, in_qsize 7, out_qsize 0
2018-03-02 20:48:39,053 : INFO : PROGRESS: at 52.55% examples, 459598 words/s, in_qsize 7, out_qsize 1
2018-03-02 20:48:40,056 : INFO : PROGRESS: at 53.25% examples, 459456 words/s, in_qsize 7, out_qsize 0
2018-03-02 20:48:41,061 : INFO : PROGRESS: at 54.00% examples, 459683 wor

2018-03-02 20:49:46,754 : INFO : saving Word2Vec object under 300features_40minwords_10context, separately None
2018-03-02 20:49:46,756 : INFO : not storing attribute syn0norm
2018-03-02 20:49:46,761 : INFO : not storing attribute cum_table
2018-03-02 20:49:47,828 : INFO : saved 300features_40minwords_10context


In [49]:
model.doesnt_match("man woman child kitchen".split())

'kitchen'

In [50]:
 model.doesnt_match("paris berlin london austria".split())

'paris'

In [51]:
 model.most_similar("man")

[('woman', 0.6322223544120789),
 ('lady', 0.6050903797149658),
 ('lad', 0.5730961561203003),
 ('farmer', 0.545566201210022),
 ('monk', 0.5349361896514893),
 ('guy', 0.5254600644111633),
 ('men', 0.5159926414489746),
 ('person', 0.5097220540046692),
 ('chap', 0.5068511962890625),
 ('businessman', 0.5058785676956177)]

In [52]:
model.most_similar("queen")

[('princess', 0.6525651216506958),
 ('belle', 0.6164785623550415),
 ('victoria', 0.600636899471283),
 ('showgirl', 0.5995155572891235),
 ('bride', 0.5839683413505554),
 ('stepmother', 0.5810116529464722),
 ('goddess', 0.5724537968635559),
 ('mistress', 0.5650918483734131),
 ('maid', 0.5603821277618408),
 ('stripper', 0.5583852529525757)]

In [53]:
 model.most_similar("awful")

[('terrible', 0.7609171867370605),
 ('horrible', 0.7466124892234802),
 ('atrocious', 0.7284055948257446),
 ('abysmal', 0.7004271745681763),
 ('dreadful', 0.695834219455719),
 ('horrid', 0.6807050704956055),
 ('horrendous', 0.6804500818252563),
 ('appalling', 0.6463396549224854),
 ('lousy', 0.630431592464447),
 ('amateurish', 0.6105203628540039)]